# The `next_tool` and `manual_call`

## Overview 

If you have worked with function calls before, you know that often the model does not call the function you want to call. This poses a knonw problem of AI Agents, because often want a particular flow to be respected. 

In this notebook, we will see how to use the `next_tool` and `manual_call` to control the flow of the AI Agent, make it more reliable and predictable.

#### next_tool

The `next_tool` is a parameter that can be used with the `function_tool` or  `agent_tool` to specify the next tool to be called. Behind the scenes, the `next_tool` is just setting the completion parameter `tool_choice` to the name of the next tool to be called. 

To use it, you just need to pass the name of of the method you want to call next as the value of the `next_tool` parameter. Note that the method must be decored as a tool and must be in the same class as the current tool.


#### manual_call

The `manual_call` is a parameter that can be used together with the `next_tool` to specify that the next tool call will be inserted "manually" in the message history. By manually, we mean that the next tool call will not be done by a new model call, but inserted directly in the message history. This is usefull in the situations where the output of the current tool can be directly used as the input of the next tool, without the need of a new model call.

To use it, you need to pass a callable as the value of the `manual_call` parameter, that will receive the output of the current tool and return a dictionay that should match the input of the next tool. In this way you can add some processing to the output of the current tool before passing it to the next tool.

---

## Let's see how to use these parameters in practice.


In the example bellow we have a simple agent that has two tools, one to generate a random topic and another that genrate a joken, given a topic.

To show that the `next_tool` is working, we will force the agent to call the `generate_joke` tool after a random topic is generated.

In [3]:
from agente.core.base import BaseAgent,BaseTaskAgent
from agente.core.decorators import function_tool,agent_tool
from typing import Annotated
import random
from dotenv import load_dotenv
load_dotenv()

True

In [4]:

class JokeTeller(BaseTaskAgent):
    agent_name: str = "JokeTeller"
    system_prompt:str = "Your task is to write a funny joke."
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": False,
    }

    @function_tool
    def task_completed(self,joke:str):
        """To be used as a tool to complete the task.

        Args:
            joke: The joke to return.
        """
        return joke



class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    @function_tool(next_tool = "get_joke") # To make sure the agent calls the get_joke tool we add the next_tool argument to force it.
    def random_topic(self):
        """Tool to get a random topic.
        """
        topics = ["programming","science","animals","food","sports"]
        topic = random.choice(topics)

        return topic


    @agent_tool()
    def get_joke(self,joke_topic:Annotated[str, "The topic of the joke"]):
        """Tool to get a joke."""

        joke_agent = JokeTeller()
        joke_agent.add_message(role = "user", content = "Tell me a joke about " + joke_topic)
        return joke_agent

In [8]:
example_agent = MainAgent()
example_agent.add_message(role = "user", content = "Generate a random topic.")
response = await example_agent.run(stream = False)

Executing agent: main_agent
Executing tool: random_topic (agent: main_agent)
Executing agent: main_agent
Executing agent: JokeTeller
Executing tool: task_completed (agent: JokeTeller)
Executing agent: main_agent


In [12]:
print(example_agent.conv_history.messages[-1].content[0].text)

The random topic is "sports."

Here's a sports joke:
Why did the basketball team go to the bank? Because they wanted to get their bounce checks!


In [13]:
for m in example_agent.conv_history.messages:
    print(m)
    print("--------------------------------")

role='user' agent_name='main_agent' content=[Content(type='text', text='Generate a random topic.')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 29, 58, 70890)
--------------------------------
role='assistant' agent_name='main_agent' content=[] tool_calls=[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_wqcvAcBqwnbcaS6We1cT0fLR', 'type': 'function'}] tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 29, 58, 605814)
--------------------------------
role='tool' agent_name='main_agent' content=[Content(type='text', text='"sports"')] tool_calls=None tool_call_id='call_wqcvAcBqwnbcaS6We1cT0fLR' tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 29, 58, 606328)
--------------------------------
role='assistant' agent_name='main_agent' content=[] tool_calls=[{'index': 0, 'function': {

An inspection to the full message history will show that the `get_joke` tool was called after the `random_topic` tool.

In [14]:
for m in example_agent.conv_history.messages:
    if m.tool_calls:
        print(m.tool_calls)

[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_wqcvAcBqwnbcaS6We1cT0fLR', 'type': 'function'}]
[{'index': 0, 'function': {'arguments': '{"joke_topic":"sports"}', 'name': 'get_joke'}, 'id': 'call_vXGyP22zQaP80OK4HTX6W0Bz', 'type': 'function'}]


And in total we add 3 model calls:

In [15]:
len(example_agent.log_calls)

3

Now, let's see how to use the `manual_call` parameter. 

We just need to add a function that will pass the generated topic to the `get_joke` tool. We will use a simple lambda function to do that.

In [16]:
class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    @function_tool(next_tool = "get_joke", manual_call= lambda x: {'joke_topic':x})
    def random_topic(self):
        """Tool to get a random topic.
        """
        topics = ["programming","science","animals","food","sports"]
        topic = random.choice(topics)

        return topic


    @agent_tool()
    def get_joke(self,joke_topic:str):
        """Tool to get a joke.

        Args:
            joke_topic: The topic of the joke.
        """

        joke_agent = JokeTeller()
        joke_agent.add_message(role = "user", content = "Tell me a joke about " + joke_topic)
        return joke_agent

In [17]:
example_agent = MainAgent()
example_agent.add_message(role = "user", content = "Generate a random topic.")
response = await example_agent.run(stream = False)

Executing agent: main_agent
Executing tool: random_topic (agent: main_agent)
Executing agent: JokeTeller
Executing tool: task_completed (agent: JokeTeller)
Executing agent: main_agent


In [19]:
for m in example_agent.conv_history.messages:
    print(m)
    print("--------------------------------")

role='user' agent_name='main_agent' content=[Content(type='text', text='Generate a random topic.')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 31, 14, 443791)
--------------------------------
role='assistant' agent_name='main_agent' content=[] tool_calls=[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_ISkA0r4SpJdIs5213YPolBI8', 'type': 'function'}] tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 31, 15, 288487)
--------------------------------
role='tool' agent_name='main_agent' content=[Content(type='text', text='"sports"')] tool_calls=None tool_call_id='call_ISkA0r4SpJdIs5213YPolBI8' tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 31, 15, 289084)
--------------------------------
role='assistant' agent_name='main_agent' content=[] tool_calls=[{'id': 'manual_QsZW85lA4p

In [20]:
for m in example_agent.conv_history.messages:
    if m.tool_calls:
        print(m.tool_calls)

[{'index': 0, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'call_ISkA0r4SpJdIs5213YPolBI8', 'type': 'function'}]
[{'id': 'manual_QsZW85lA4pfpmz155dpLiYme', 'function': {'name': 'get_joke', 'arguments': '{"joke_topic": "sports"}'}, 'type': 'function'}]


The manual call has the prefix "man_" to indicate that the tool call was inserted manually in the message history and not by a new model call.

We can see in the call log that we now have only 2 model calls.

In [21]:
len(example_agent.log_calls)

2